### Setup

In [1]:
# Required for data reading & manipulation
import pandas as pd

# Required for JSON file path
from os import path

import folium

import matplotlib.pyplot as plt
%matplotlib inline

### Initial import

In [2]:
# Read the 'train' data (JSON file) downloaded from Kaggle

directory = 'data'
file_name = 'train.json'

data = pd.read_json(path.join(directory, file_name), convert_dates=['created'])

In [3]:
data.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [4]:
# Resetting index. Original indexing is messy.
data.reset_index(drop=True, inplace=True)

In [5]:
data.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address'],
      dtype='object')

In [6]:
data.dtypes

bathrooms                 float64
bedrooms                    int64
building_id                object
created            datetime64[ns]
description                object
display_address            object
features                   object
interest_level             object
latitude                  float64
listing_id                  int64
longitude                 float64
manager_id                 object
photos                     object
price                       int64
street_address             object
dtype: object

In [7]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
bathrooms,49352.0,1.212180e+00,0.501420,0.000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+01
bedrooms,49352.0,1.541640e+00,1.115018,0.000,1.000000e+00,1.000000e+00,2.000000e+00,8.000000e+00
latitude,49352.0,4.074154e+01,0.638535,0.000,4.072830e+01,4.075180e+01,4.077430e+01,4.488350e+01
listing_id,49352.0,7.024055e+06,126274.611244,6811957.000,6.915888e+06,7.021070e+06,7.128733e+06,7.753784e+06
longitude,49352.0,-7.395572e+01,1.177912,-118.271,-7.399170e+01,-7.397790e+01,-7.395480e+01,0.000000e+00
price,49352.0,3.830174e+03,22066.865885,43.000,2.500000e+03,3.150000e+03,4.100000e+03,4.490000e+06


In [8]:
len(data['listing_id'].unique())

49352

In [9]:
# Describe objects in data. Note 'features' and 'photos' have been dropped due to being list types.
data.drop(['features','photos'], axis=1).describe(include=['O']).T

,count,unique,top,freq
building_id,49352,7585,0,8286
description,49352,38244,,1647
display_address,49352,8826,Broadway,438
interest_level,49352,3,low,34284
manager_id,49352,3481,e6472c7237327dd3903b3d6f6a94515a,2533
street_address,49352,15358,3333 Broadway,174


In [10]:
# Describe 'features' column
data['features'].apply(tuple).describe(include='O').T

count     49352
unique    10254
top          ()
freq       3218
Name: features, dtype: object

In [11]:
# Describe 'photos' column
data['photos'].apply(tuple).describe(include='O').T

count     49352
unique    45677
top          ()
freq       3615
Name: photos, dtype: object

In [12]:
data.loc[10,'photos']

['https://photos.renthop.com/2/6869199_06b2601f053d04ca5a9b19d477b7370d.jpg']

In [13]:
# Check for missing data
data.isnull().sum()

bathrooms          0
bedrooms           0
building_id        0
created            0
description        0
display_address    0
features           0
interest_level     0
latitude           0
listing_id         0
longitude          0
manager_id         0
photos             0
price              0
street_address     0
dtype: int64

### Remove records with price outliers

In [14]:
def print_listing_mask(data, mask, cols):
    print('--- TOTAL RESULTS: ', mask.sum(), '---\n')
    
    if not set(cols) <= set(data.columns):
        print('Warning: nonexistent columns provided. Columns {} will be ignored'.format(list(set(cols).difference(set(data.columns)))))
    
    cont = input('Print (y/n)?: ')
    
    if cont.lower() == 'y':
        print('-'*10)
        for i, row in data[mask].iterrows():
            
            print('INDEX: {}'.format(i))
            for col in cols:
                try:
                    print('{}: {}'.format(col.upper(),row[col]))
                except:
                    pass
            print('-'*10)

In [15]:
mask_price = data['price'] > 30000 # Initial analysis of most expensive listings
display_cols = ['description','price', 'bedrooms', 'bathrooms']

print_listing_mask(data, mask_price, display_cols)

--- TOTAL RESULTS:  35 ---

Print (y/n)?: y
----------
INDEX: 2743
DESCRIPTION: Old World Deco charm in this  large apartment  perfect for a family or equally ideal for the individual or couple looking for a home on the Park.  Situated in NYC's most desirable location, just steps from Lincoln Center<br /><br />Approximately 4000 Sq. Ft at this Historic CPW  location. 14' ceilings and 3 fireplaces with magnificent mantles. Restored original details.Windowed kitchen apartment In-wall air conditioning & climate control, new windows. Museum quality details throughout with Mahogany and Oak Wainscoting. Den features floor to ceiling bookshelves. Living room and library can be closed for privacy by exquisite original pocket doors. 24-hour doorman and live-in superintendent.<br /><br />(Photos representational only)Call for details and to view<br /><br /><br /><br /><br /><br /><p><a  website_redacted 
PRICE: 35000
BEDROOMS: 4
BATHROOMS: 4.0
----------
INDEX: 5250
DESCRIPTION: Remarkable Revam

In [16]:
# Based of analysis of above results (index: reasoning)
price_outliers_50k = {8042: 'commercial building',
                      9590: 'insufficient info to justify price',
                      10581: 'full building',
                      18735: 'insufficient info to justify price',
                      19558: 'insufficient info to justify price',
                      25538: 'insufficient info to justify price',
                      29665: 'insufficient info to justify price',
                      30689: 'too expensive for 1 bed/1 bath rental',
                      44832: 'retail'}

In [17]:
def drop_outlier_dict(data, outlier_dict, inplace=False, display=False):
    
    original_size = len(data)
    
    if inplace:
        d = data
    else:
        d = data.copy()
    
    n_outliers = len(outlier_dict)
    n_errors = 0
    
    for index in outlier_dict:
        try:
            d.drop(index, inplace=True)
        except Exception as e:
            n_errors += 1
            print(e)
    
    if display:
        error_summary = '' if n_errors == 0 else ' ({} errors)'.format(n_errors)
        print('Original size: {}'.format(original_size))
        print('{}/{} records removed.{}'.format(n_outliers-n_errors, n_outliers, error_summary))
        print('Remaining: {}'.format(len(d)))
    
    if not inplace:
        return d

In [18]:
# Drop price outliers identified in price_outliers_50k dictionary

drop_outlier_dict(data, price_outliers_50k, inplace=True, display=True);

Original size: 49352
9/9 records removed.
Remaining: 49343


In [19]:
mask_0bed = (data['price'] > 3000) & (data['bedrooms'] == 0) & (data['description'].str.lower().str.contains('retail', 'commercial'))

print_listing_mask(data, mask_0bed, display_cols)

--- TOTAL RESULTS:  33 ---

Print (y/n)?: y
----------
INDEX: 4098
DESCRIPTION: Prime Soho Location, just off Spring Street.  Cute Coffee Shop Retail space Sullivan/Spring Barber Salon Wine Bar.Across from a new large condo development.<br /><br />600 Square Feet<br /><br />$200/RSF<br /><br />$10,000/month<p><a  website_redacted 
PRICE: 10000
BEDROOMS: 0
BATHROOMS: 1.0
----------
INDEX: 4517
DESCRIPTION: This building is located in midtown Manhattan, near Park Avenue and the East River. The newly renovated interiors feature hardwood floors, granite countertops, stylish white-on-white cabinetry, energy-efficient stainless steel appliances and environmentally friendly finishes. Enjoy incredible river and Manhattan views from your private balcony. Walk to the Empire and Chrysler building, catch the train from Grand Central or ride with the new Citi Bike bike share station located right outside our door. We are located in historic Turtle Bay, with vibrant multi-cultural events, markets, s

In [20]:
# Based off analysis of descriptions above
commercial_outliers = {4098: 'coffee shop',
                      4843: 'retail',
                      5613: 'retail',
                      15189: 'commercial'}

In [21]:
drop_outlier_dict(data, commercial_outliers, inplace=True, display=True)

Original size: 49343
4/4 records removed.
Remaining: 49339


In [22]:
mask_lowprice = data['price'] < 1000

print_listing_mask(data, mask_lowprice, display_cols)

--- TOTAL RESULTS:  19 ---

Print (y/n)?: y
----------
INDEX: 1236
DESCRIPTION:         
PRICE: 695
BEDROOMS: 0
BATHROOMS: 1.0
----------
INDEX: 4020
email, call, text anytimencluded tment for short term with option to stay 5/15- 9/1 
PRICE: 800
BEDROOMS: 1
BATHROOMS: 1.0
----------
INDEX: 5942
DESCRIPTION:         
PRICE: 695
BEDROOMS: 0
BATHROOMS: 1.0
----------
INDEX: 5966
DESCRIPTION:         
PRICE: 695
BEDROOMS: 0
BATHROOMS: 1.0
----------
INDEX: 6850
DESCRIPTION:         
PRICE: 999
BEDROOMS: 0
BATHROOMS: 1.0
----------
INDEX: 7654
DESCRIPTION: Now available, absolutely stunning 1 bedroom 1 bathroom apartment located in the heart of the Upper West Side! Apartment is located in a 24-hour concierge, doorman/elevator building with a laundry facility on site! Unit features gorgeous open living room layout with good natural light, massive king sized bedroom, full size kitchen appliances including a dishwasher, updated cabinetry with tons of storage space, walk-in closets, high ceilin

In [23]:
# Based off analysis of descriptions above
lowprice_outliers = {28626: '43/mo, no info',
                     43824: '45/mo, no info'}

In [24]:
drop_outlier_dict(data, lowprice_outliers, inplace=True, display=True)

Original size: 49339
2/2 records removed.
Remaining: 49337


### Remove records with missing/irrelevent coordinates

In [25]:
missing_coord_mask = (data['latitude'] == 0) | (data['longitude'] == 0)

coord_cols = display_cols + ['latitude', 'longitude']

print_listing_mask(data, missing_coord_mask, coord_cols)

--- TOTAL RESULTS:  12 ---

Print (y/n)?: y
----------
INDEX: 2787
DESCRIPTION: ***NO BROKER FEE***<BR><BR>Situated on a gorgeous tree-lined block, this landmarked, limestone home features the perfect balance of old-world charm and ultra-modern, luxe finishes.<BR><BR> A one-of-a-kind, stunning kitchen anchors this home and features posh amenities like a full-width sliding glass Nanawall overlooking the immaculate backyard and patio, a separate prep area equipped with two wine coolers and a massive pantry for storage. Top-of-the-line appliances from the likes of Bertazzoni and Bosch have been perfectly paired with the highest quality Caesarstone countertops and an abundance of sleek custom cabinetry.<BR><BR> Beautiful bay windows line both the front and back of this home and let natural light pour in and shine upon lustrous hardwood floors throughout. Glide through gorgeous pocket doors in the great room and feast your eyes upon the spacious sitting room and living room, perfect for you

In [26]:
missing_coord = {index: 'missing latitude/longitude' for index in data[missing_coord_mask].index}
missing_coord

{2787: 'missing latitude/longitude',
 14104: 'missing latitude/longitude',
 21715: 'missing latitude/longitude',
 22471: 'missing latitude/longitude',
 25860: 'missing latitude/longitude',
 27181: 'missing latitude/longitude',
 33286: 'missing latitude/longitude',
 34390: 'missing latitude/longitude',
 35826: 'missing latitude/longitude',
 36448: 'missing latitude/longitude',
 39199: 'missing latitude/longitude',
 42987: 'missing latitude/longitude'}

In [27]:
drop_outlier_dict(data, missing_coord, inplace=True, display=True)

Original size: 49337
12/12 records removed.
Remaining: 49325


In [28]:
lat_mean = data['latitude'].mean()
lat_std = data['latitude'].std()

lon_mean = data['longitude'].mean()
lon_std = data['longitude'].std()

coord_outlier_3std_mask = (abs(data['latitude']-lat_mean) > 3*lat_std) | (abs(data['longitude']-lon_mean) > 3*lon_std)
coord_outlier_2std_mask = ~coord_outlier_3std_mask & ((abs(data['latitude']-lat_mean) > 2*lat_std) | (abs(data['longitude']-lon_mean) > 2*lon_std))

In [29]:
print_listing_mask(data, coord_outlier_3std_mask, coord_cols)

--- TOTAL RESULTS:  38 ---

Print (y/n)?: y
----------
INDEX: 159
DESCRIPTION: When you choose Infinity, you'll move into an exclusive neighborhood in one of the best communities in America. Our choice residences, and the elevated lifestyle they provide, compare to what you'll find in New York, San Francisco or Miami. A major financial center in its own right, Stamford is consistently ranked one of the best cities to live in the country and along with nearby Greenwich, boasts some of the finest restaurants on the East Coast.Our one and two bedroom rental homes include deluxe features that bring life to the next level. Open living and dining areas, floor-to-ceiling windows and spacious balconies provide our residents with an elevated lifestyle above and beyond expectations.Please call Leasing Office at 747-575-4675, email kagglemanager@renthop.com, or visit  website_redacted today for more information!Other Building Amenities:Rooftop lounge and fire pits.Rooftop pool.Dog washing station

Can already see in descriptions above that some listings are outside of NYC / NY State

In [30]:
# Too many to print
print_listing_mask(data, coord_outlier_2std_mask, coord_cols)

--- TOTAL RESULTS:  395 ---

Print (y/n)?: n


Will plot the 3 std and 2 std locations using folium to pick out which are actually outliers and not in NYC:

In [31]:
coord_outliers_3std = data[coord_outlier_3std_mask]
coord_outliers_2std = data[coord_outlier_2std_mask]

nyc_coords = [40.75, -73.9]
m = folium.Map(nyc_coords, zoom_start=9, tiles='CartoDB positron')

for i, row in coord_outliers_3std.iterrows():
    marker = folium.CircleMarker([row['latitude'],row['longitude']], radius=5, color='red', fill_color='white', popup=str(i))
    marker.add_to(m)

for i, row in coord_outliers_2std.iterrows():
    marker = folium.CircleMarker([row['latitude'],row['longitude']], radius=5, color='orange', fill_color='white',popup=str(i))
    marker.add_to(m)

m.choropleth(geo_path='data/NYC_Buroughs.json', fill_color='white', fill_opacity=0)

directory = 'maps'
file_name = 'coord_outliers_potential.html'
m.save(path.join(directory, file_name))

In [36]:
valid_3std = [41736, 34329, 34200] # Three markers on top of one another on Staten Island

coord_outliers_actual = coord_outliers_3std.drop(valid_3std)

invalid_2std = [45139, 4179] # Leaving the few markers that are outside the Bronx

coord_outliers_actual = coord_outliers_actual.append(coord_outliers_2std.loc[invalid_2std])

m = folium.Map(nyc_coords, zoom_start=9, tiles='CartoDB positron')

for i, row in coord_outliers_actual.iterrows():
    marker = folium.CircleMarker([row['latitude'],row['longitude']], radius=5, color='red', fill_color='white', popup=str(i))
    marker.add_to(m)

m.choropleth(geo_path='data/NYC_Buroughs.json', fill_color='white', fill_opacity=0)
    
directory = 'maps'
file_name = 'coord_outliers_actual.html'
m.save(path.join(directory, file_name))

In [37]:
coord_outlier_dict = {index: 'coords ({},{}) outside NYC'.format(row['latitude'],row['longitude']) for index, row in coord_outliers_actual.iterrows()}
coord_outlier_dict

{159: 'coords (41.0412,-73.54) outside NYC',
 2150: 'coords (39.8395,-86.1527) outside NYC',
 3925: 'coords (40.1159,-74.6267) outside NYC',
 4179: 'coords (40.6616,-74.6637) outside NYC',
 4236: 'coords (41.0411,-73.5423) outside NYC',
 6153: 'coords (40.045,-75.5214) outside NYC',
 6546: 'coords (42.2019,-70.9846) outside NYC',
 7620: 'coords (42.2509,-71.006) outside NYC',
 9694: 'coords (39.7996,-74.6248) outside NYC',
 12387: 'coords (44.6038,-75.1773) outside NYC',
 13095: 'coords (40.9697,-72.1336) outside NYC',
 13118: 'coords (41.0868,-73.8602) outside NYC',
 13234: 'coords (43.0346,-76.6336) outside NYC',
 13613: 'coords (41.0411,-73.5423) outside NYC',
 16072: 'coords (42.3459,-71.0794) outside NYC',
 16253: 'coords (41.0411,-73.5423) outside NYC',
 22437: 'coords (42.2019,-70.9846) outside NYC',
 22767: 'coords (34.0126,-118.271) outside NYC',
 23283: 'coords (42.2019,-70.9846) outside NYC',
 23302: 'coords (40.5065,-74.4708) outside NYC',
 25973: 'coords (42.8724,-73.4662)

In [34]:
drop_outlier_dict(data, coord_outlier_dict, inplace=True, display=True)

Original size: 49325
37/37 records removed.
Remaining: 49288


### Save cleaned data as new file

In [35]:
directory = 'data'
file_name = 'NYC_RealEstate_Data.json'

data.to_json(path.join(directory, file_name))